# The TinyALU Model and Initial Testbench

## TinyALU Operations

In [ ]:
import enum

@enum.unique
class Ops(enum.IntEnum):
    """Legal ops for the TinyALU"""
    ADD = 1
    AND = 2
    XOR = 3
    MUL = 4

## The Simplest TinyALU TLM Model

In [2]:
import enum

@enum.unique
class Ops(enum.IntEnum):
    """Legal ops for the TinyALU"""
    ADD = 1
    AND = 2
    XOR = 3
    MUL = 4

import time

def alu_op(A, B, op):
    """Python model of the TinyALU"""
    assert isinstance(op, Ops), "The tinyalu op must be of type ops"
    if op == Ops.ADD:
        result = A + B
    elif op == Ops.AND:
        result = A & B
    elif op == Ops.XOR:
        result = A ^ B
    elif op == Ops.MUL:
        result = A * B
    time.sleep(0.1)
    return result

print(f"5 + 8 = {alu_op(5,8,Ops.ADD)}")
print(f"0xF0 & 0xFF = 0x{alu_op(0xF0, 0xFF, Ops.AND):02x}")
print(f"0x5 ^ 0xA = 0x{alu_op(5,0xA, Ops.XOR):02x}")
print(f"10 * 10 = {alu_op(10, 10, Ops.MUL)}")

5 + 8 = 13
0xF0 & 0xFF = 0xf0
0x5 ^ 0xA = 0x0f
10 * 10 = 100


## A Simple Constrained Random Testbench

Add an error into the TinyALU Model and test the testbench.

In [8]:
import enum

@enum.unique
class Ops(enum.IntEnum):
    """Legal ops for the TinyALU"""
    ADD = 1
    AND = 2
    XOR = 3
    MUL = 4

import time
import random

def alu_op(A, B, op, error = False):
    """Python model of the TinyALU"""
    assert isinstance(op, Ops), "The tinyalu op must be of type ops"
    if op == Ops.ADD:
        result = A + B
    elif op == Ops.AND:
        result = A & B
    elif op == Ops.XOR:
        result = A ^ B
    elif op == Ops.MUL:
        result = A * B
    time.sleep(0.1)
    if error and (random.randint(0,3) == 0):
        result = result + 1
    return result

import logging
logging.basicConfig(level=logging.NOTSET)
logger = logging.getLogger()
cvg = set() #functional coverage
for _ in range(3):
    A = random.randrange(256)
    B = random.randrange(256)
    op = random.choice(list(Ops))
    cvg.add(op)
    predicted_result = alu_op(A, B, op)
    actual_result = alu_op(A, B, op, error=True)
    if predicted_result == actual_result:
        logger.info (f"PASSED:  {A:02x} {op.name} {B:02x} = {actual_result:04x}")
    else:
        logger.error (f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")
if len(set(Ops) - cvg) > 0:
   logger.error(f"Functional coverage error. Missed: {set(Ops)-cvg}")

ERROR:root:FAILED: b3 AND 6e = 0023 expected 0022
INFO:root:PASSED:  c8 ADD e4 = 01ac
INFO:root:PASSED:  73 ADD 0b = 007e
ERROR:root:Functional coverage error. Missed: {<Ops.XOR: 3>, <Ops.MUL: 4>}
